# ANALIZA PODATKOV

In [8]:
import pandas as pd
import analiza_funkcije

filmi = pd.read_csv("filmi.csv")

In [9]:
filmi["razlika_v_ocenah"] = abs(filmi["ocena_rotten_tomatoes"] / 10 - filmi["ocena_imdb"])

In [10]:
filmi["desetletje"] = (filmi["leto"] // 10) * 10

In [11]:
filmi["visja_ocena"] = filmi.apply(analiza_funkcije.doloci_visjo_oceno, axis=1)

In [ ]:
#za narediti:
#katero leto ima koliko filmov